In [1]:
from datasets import load_dataset,Dataset
dataset = load_dataset("squad_kor_v1")

Found cached dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import concatenate_datasets,Dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [7]:
import pandas as pd
tmp = pd.DataFrame(dataset['train'])

In [9]:
type(tmp['answers'][0])

dict

In [19]:
a = Dataset.from_pandas(tmp)

In [11]:
from datasets import load_from_disk
df = load_from_disk('/opt/ml/input/data/train_dataset/')
df = pd.DataFrame(df['train'])
type(df['answers'][0])

dict

In [18]:
b = Dataset.from_pandas(df)

In [20]:
concatenate_datasets([a,b])

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 64359
})

In [25]:
from datasets import DatasetDict
def aug_data(data, new_data):
    assert type(new_data) == DatasetDict, f'it mustnbe a Dataset type but you give {type(new_data)}'
    train_df = Dataset.from_pandas(pd.DataFrame(new_data['train']))
    validation_df = Dataset.from_pandas(pd.DataFrame(new_data['validation']))
    train_data = concatenate_datasets([data['train'],train_df])
    validation_data = concatenate_datasets([data['validation'],validation_df])
    aug_dataset = DatasetDict({'train': train_data, 'validation' : validation_data})
    return aug_dataset

In [1]:
df = load_from_disk('/opt/ml/input/data/train_dataset/')
dataset = load_dataset("squad_kor_v1")

NameError: name 'load_from_disk' is not defined

In [27]:
concat_df = aug_data(df, dataset)

In [28]:
concat_df

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 64359
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 6014
    })
})

In [64]:
dataset['train'].features['answers'] # korquad

Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)

In [55]:
df['train'].features['answers']   # competiotion MRC

{'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [63]:
dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 60407
})

In [62]:
from datasets import Sequence
def answer_change(example):
    example['answers'] = {'answer_start' : Sequence(feature = example.features['answers'].feature['answer_start']),
                        'text' : Sequence(feature = example.features['answers'].feature['text'])
                            }
    return example
dataset['train'].map(answer_change)

  0%|          | 0/60407 [00:00<?, ?ex/s]

KeyError: 'answers'

In [39]:
df['train'].features['answers']

{'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [35]:
df =df['train'].remove_columns(['document_id', '__index_level_0__'])

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [59]:
a = list(df['train'].remove_columns(['document_id', '__index_level_0__']).features)

In [62]:
dataset['train'].features([a])

TypeError: 'Features' object is not callable